In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
utils_ops.tf = tf.compat.v1
tf.gfile = tf.io.gfile

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [ ]:
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [ ]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]

  output_dict = model(input_tensor)

  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  if 'detection_masks' in output_dict:
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
def show_inference(model, image_path):
    image_np = image_path
    image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
    output_dict = run_inference_for_single_image(model, image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2)
    image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
    bicycle_detect(image_np,output_dict['detection_classes'],output_dict['detection_scores'],output_dict['detection_boxes'])
    #return image_np

In [ ]:
from datetime import datetime

def bicycle_detect(image,classes,score,boxes):
    
    for i in range(10):
        if(classes[i]==2 and score[i]>0.8):
            
            h,w=image.shape[0:2]
            
            ymin,xmin,ymax,xmax=boxes[i]

            now = datetime.now()
            dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
            
            center=(int(((xmin+xmax)/2)*w),int(((ymin+ymax)/2)*h))
            cv2.circle(image,center,10,(0,0,255),-1)
            
            file_name=os.path.join('E:/TEST/',dt_string+'.jpg')
            cv2.imwrite(file_name,image)

In [ ]:
import cv2
import numpy as np
import tkinter

#line detection codes strt

x=100
video=cv2.VideoCapture(r'F:\research full codes\Samples\Comp 7.mp4')



width=480
fcx=width//2


kernel=np.array([[-1,0,1],
                 [-2,0,2],
                 [-1,0,1]],np.float)



while(True):
    ret,imgs=video.read()
    img=cv2.resize(imgs,(480,340))
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    blur=cv2.blur(gray,(5,5))
    
    conv=cv2.filter2D(blur,-1,kernel)
    
    img[:,fcx]=(0,255,0)
    
    
    for i in range(240):
        for j in range(240-(i),0,-1):
            conv[i,j]=0
            gray[i,j]=0
    
    
    
    
    ret, thresh = cv2.threshold(conv, 50, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        area=cv2.contourArea(cnt)
        #print(area)
        if(area>300 and area>310):
            cv2.drawContours(img, [cnt], -1, (0,0,255), 3)            
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cv2.drawContours(img,[box],0,(0,255,0),2)
            
            
            M=cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            cv2.circle(img,(cx,cy),5,(0,255,255),-1)
            
            if(cx<fcx and cy>150):
                cv2.line(img,(cx,cy),(fcx,cy),(255,0,0,),2)
                distance=fcx-cx
                cv2.putText(img,"distance: "+str(distance),(fcx,170),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
                
            #if(distance<=150):
                    #print ("Distance is : "+str(distance))
            
    
    img=show_inference(detection_model,img)
    cv2.imshow('LIVE',img)
    #cv2.imshow('thresh',thresh)
    #cv2.imshow('gray',gray)
    key=cv2.waitKey(1)

    if(key==27):
        break
    
    
   # img=show_inference(detection_model,img)
   # cv2.imshow('LIVE',img)
   # k=cv2.waitKey(1)
    
    #if(k==27):
    #    break
#cv2.destroyAllWindows()
cv2.destroyAllWindows()
source.release()